<a href="https://colab.research.google.com/github/lakshitgosain/Tensorflow-ZTM/blob/main/TF_ZTM_07_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Fundamentals in TF

Sequence-to-sequence
* One to one- Image Captioning
* Many to one- Sentiment analysis
* Many to one- time series forecasting
* many- to-many- Machine translation


In [2]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-f5d0f1aa-2c9c-3177-0a7a-37c73e05b81f)


In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-06-15 09:34:44--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-06-15 09:34:44 (125 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [5]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

--2023-06-15 09:34:47--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.6.128, 108.177.112.128, 74.125.124.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.6.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2023-06-15 09:34:47 (155 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [6]:
unzip_data("nlp_getting_started.zip")

## Visualize the dataset

To visualize the sample data, we need to read the in. On way is to use python.

Another way to do this is use pandas



In [7]:
import pandas as pd
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")

In [8]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
train_data_shuffled=train_df.sample(frac=1, random_state=42)

In [10]:
train_data_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [11]:
#Test dataframe looks like:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [12]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [13]:
#How many total number of samples
len(train_df), len(test_df)

(7613, 3263)

In [14]:
#Let's visualize some random training samples
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_data_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _,text, target=row
  print(f"Target: {target}", "(real disaster)" if target >0 else "(not real disaster)")
  print(f"text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
text:
@_gaabyx we got purple activist I thought it was a drought

---

Target: 0 (not real disaster)
text:
@BuffoonMike I knew mo not doing much would bite us he was influenced by that shitty staff and injuries are not acquisitions

---

Target: 0 (not real disaster)
text:
@toddstarnes Enjoy the impending landslide Todd. Hehe.

---

Target: 0 (not real disaster)
text:
GO LOOK AT GRIZZLY PEAK RIGHT NOW... It looks like the beginning of an dystopian apocalypse movie

---

Target: 0 (not real disaster)
text:
Ignition Knock (Detonation) Sensor-Senso Standard KS100 http://t.co/7o4lNfBe7K http://t.co/fVZSGJtBew

---



### Split data into training and validation sets



In [15]:
from sklearn.model_selection import train_test_split

In [16]:
train_sentences, val_sentences, train_labels, val_labels=train_test_split(train_data_shuffled['text'].to_numpy(),
                                                                          train_data_shuffled['target'].to_numpy(),
                                                                          test_size=0.1,
                                                                          random_state=42)

In [17]:
#Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

## Converting words into numbers

First thing while building a model is to convert your text into numbers


Tokenization VS Embedding
Tkenization - word level and character level. Direct apping of a token.

Embedding - every word gets turned into a vector and we can define size of the vector. Embeddings can learn as our model trains



Text vectorization(tokenization)

In [18]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [19]:
#Use the default text vectorization parameters
text_vectoizer= TextVectorization(max_tokens=None, #Defines how many words in our vocab
                               standardize= "lower_and_strip_punctuation",
                               split="whitespace",
                               ngrams=None,
                               output_mode='int',
                               output_sequence_length=None) #how long do you want your sequences to be
                               #pad_to_max_tokens=True)

In [20]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [21]:
#Find the average number of tokens(words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [22]:
max_vocab_length= 10000 #max no of words to have in our vocab
max_length=15 #Max length our sequences will be (e.g. how many words from a Tweet does a model see)

text_vectorizer=TextVectorization(max_tokens=max_vocab_length,
                                     output_mode="int",
                                     output_sequence_length=max_length)

In [23]:
#Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)


In [24]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

In [25]:
#Create a sample sentence and tokenize it
sample_sentence="There is a floow in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3,   1,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [26]:
random_sentence=random.choice(train_sentences)
print(random_sentence)
vectorized_random_sentence=text_vectorizer([random_sentence])
print(f"Random Sentence: {random_sentence}\nVectoized Random Sentence: {vectorized_random_sentence}")

If Flooding occurs in your home - How to stop mold growth via @ProudGreenHome http://t.co/KAVAovJz2V
Random Sentence: If Flooding occurs in your home - How to stop mold growth via @ProudGreenHome http://t.co/KAVAovJz2V
Vectoized Random Sentence: [[  47  231 3575    4   33  153   62    5  240    1 3800   49 9518    1
     0]]


In [27]:
words_in_vocab=text_vectorizer.get_vocabulary()
top_5_words=words_in_vocab[:5]
botton_5_words=words_in_vocab[-5:]

In [28]:
words_in_vocab,top_5_words,botton_5_words

(['',
  '[UNK]',
  'the',
  'a',
  'in',
  'to',
  'of',
  'and',
  'i',
  'is',
  'for',
  'on',
  'you',
  'my',
  'with',
  'it',
  'that',
  'at',
  'by',
  'this',
  'from',
  'be',
  'are',
  'was',
  'have',
  'like',
  'as',
  'up',
  'so',
  'just',
  'but',
  'me',
  'im',
  'your',
  'not',
  'amp',
  'out',
  'its',
  'will',
  'an',
  'no',
  'has',
  'fire',
  'after',
  'all',
  'when',
  'we',
  'if',
  'now',
  'via',
  'new',
  'more',
  'get',
  'or',
  'about',
  'what',
  'he',
  'people',
  'news',
  'been',
  'over',
  'one',
  'how',
  'dont',
  'they',
  'who',
  'into',
  'were',
  'do',
  'us',
  '2',
  'can',
  'video',
  'emergency',
  'there',
  'disaster',
  'than',
  'police',
  'would',
  'his',
  'still',
  'her',
  'some',
  'body',
  'storm',
  'crash',
  'burning',
  'suicide',
  'back',
  'man',
  'california',
  'why',
  'time',
  'them',
  'had',
  'buildings',
  'rt',
  'first',
  'cant',
  'see',
  'got',
  'day',
  'off',
  'our',
  'going',
 

### Creating an embedding using an embedding layer

To make our embedding, we're going to use the TF's embedding layer.

The parameters we care the most about our embedding layers are:
* input_dim= sie of our vocab
* output_dim= size of the output embedding vector. A value of a 100 will mean that each token will be represented as a vector of 100 long length
* input_length= length of sequences being passed to the embedding layer
*

In [29]:
from tensorflow.keras import layers

In [30]:
embedding=layers.Embedding(input_dim=max_vocab_length,
                           output_dim=128,
                           input_length=max_length)

In [31]:
embedding

In [32]:
#Get a random sentence from the training set
random_sentence=random.choice(train_sentences)
print(f"Original Sentence:{random_sentence}\
      \nEmbedded Sentence")

sample_embedding=embedding(text_vectorizer([random_sentence]))
sample_embedding

Original Sentence:why wasn't this warship sunk?      CNN: First on CNN: Iranian warship points weapon at U.S. helicopter official says http://t.co/hfZk09McEN      
Embedded Sentence


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00148789, -0.04278181,  0.04509476, ...,  0.02537778,
          0.03929676,  0.00520863],
        [ 0.00253608, -0.04308238,  0.04144745, ...,  0.02313714,
          0.02125427, -0.02178888],
        [ 0.0011446 , -0.02509735,  0.03938497, ..., -0.03540399,
         -0.01956787,  0.01895989],
        ...,
        [ 0.02632589, -0.00197272, -0.03334183, ..., -0.03353756,
          0.0323332 ,  0.00638876],
        [-0.04334159,  0.01059445, -0.00653569, ...,  0.02808512,
          0.00860498, -0.00653278],
        [ 0.02961126, -0.02270567, -0.03333702, ...,  0.04598988,
         -0.01909754,  0.03559836]]], dtype=float32)>

In [33]:
#Check out a single token's embedding
sample_embedding[0][0], sample_embedding[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.00148789, -0.04278181,  0.04509476,  0.04089579,  0.00330998,
         0.04700874, -0.04065471, -0.01724187, -0.01614218,  0.03740994,
         0.02866549, -0.00691136, -0.01156968,  0.00551845,  0.04392344,
         0.0448716 ,  0.01411655,  0.0105685 , -0.03000736,  0.00587998,
         0.04086921,  0.01294992, -0.02317032, -0.00826484,  0.01717777,
        -0.0030687 , -0.02133669,  0.01795919, -0.03354864,  0.0085291 ,
        -0.02273786,  0.00186324,  0.0287887 , -0.01754604, -0.03441245,
         0.02943324, -0.04534518, -0.04160284, -0.0275494 , -0.02335023,
         0.00099506, -0.03505803, -0.04095988,  0.02528736,  0.003339  ,
         0.02731163,  0.01783302,  0.02046027, -0.00030277, -0.01723192,
         0.02199015,  0.01091851, -0.01285362,  0.04649189, -0.04768066,
        -0.00230713,  0.02154497,  0.0453601 ,  0.00959893, -0.035699  ,
         0.00300832,  0.03540437,  0.02942953, -0.03339862, -0.02019421,
  

##Modelling a text dataset(running a series of experiments)

* Model 0: Naive Bayes with Tokenization
* Model 1: Feed-forward Neural Network(Dense Model)
* Model 2: LSTM Model(RNN)
* Model 3: GRU Model
* Moel 4: Bidirectional LSTM
* Model 5 : 1D Convolutional Layer
* model 6: Tensorflow hub pretrained feature extraction (using transfer learning)
* Model 7 : Same as model 6 with 10% of data

How are we going to approach all of these:
Use the standard steps in modelling with tensorflow

* Create->build->fit->Evaluate



### Model 0

To create a baseline, we'll use sklearn's Naive Bayes using Tf-IDF to convert our words to numbers

It is a good practice to use non-DL Algos as a baseline because of their speed and then later use DL to see if you can improve upon them

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [37]:
model_0=Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [38]:
#Evaluate the baseline Model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"our baseline model achieves an accuracy of : {baseline_score*100:.2f}%")

our baseline model achieves an accuracy of : 79.27%


In [41]:
train_df.target.value_counts

<bound method IndexOpsMixin.value_counts of 0       1
1       1
2       1
3       1
4       1
       ..
7608    1
7609    1
7610    1
7611    1
7612    1
Name: target, Length: 7613, dtype: int64>

In [43]:
#Make predictions
baseline_preds= model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Create an evaluation function for our model experiments

We could evaluate all of our model's predictions with different metrics every time.
Calculate:
* Accuracy
* Precision
* Recall
* F1 score


In [46]:
#Function to evaluate the above metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculated model accuracy, precision, recall and f1 score of a binary classification model
  """
  #Calculate model accuracy
  model_accuracy=accuracy_score(y_true, y_pred) * 100
  #Calculate model precision, recall and f1 score using the"weighted average"
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred,average="weighted" )
  model_results={'accuracy':model_accuracy,
                 'precision': model_precision,
                 'recall': model_recall,
                 'f1-score':model_f1}
  return model_results

In [47]:
#Get baseline results
baseline_results=calculate_results(y_true=val_labels,
                                   y_pred=baseline_preds)

In [48]:
baseline_results['accuracy']

79.26509186351706

In [50]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1-score': 0.7862189758049549}

## Model 1: A simple Dense Model

In [52]:
#Create a tensorflow callback
from helper_functions import create_tensorboard_callback

#Create a directory to save tensorboard logs
SAVE_DIR="model_logs"


In [68]:
#Build model with functional API
from tensorflow.keras import layers

inputs=layers.Input(shape=(1,), dtype=tf.string) #Inputs are 1 dimensional
x= text_vectorizer(inputs) #Turn the input text into numbers
x= embedding(x) #Create an embedding of vctorized embeddings
x=layers.GlobalAveragePooling1D()(x)
outputs=layers.Dense(1, activation='sigmoid')(x)

model_1=tf.keras.Model(inputs, outputs, name="Model_1_dense")

In [69]:
model_1.summary()

Model: "Model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [70]:
#Compile model
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [71]:
model_1_history=model_1.fit(x=train_sentences,
                            y=train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                   experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230615-103643
Epoch 1/5
215/215 [==============================] - 20s 70ms/step - loss: 0.6110 - accuracy: 0.6879 - val_loss: 0.5389 - val_accuracy: 0.7454
Epoch 2/5
215/215 [==============================] - 2s 12ms/step - loss: 0.4418 - accuracy: 0.8156 - val_loss: 0.4686 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3471 - accuracy: 0.8610 - val_loss: 0.4558 - val_accuracy: 0.7927
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2846 - accuracy: 0.8911 - val_loss: 0.4632 - val_accuracy: 0.7913
Epoch 5/5
215/215 [==============================] - 2s 7ms/step - loss: 0.2381 - accuracy: 0.9111 - val_loss: 0.4785 - val_accuracy: 0.7913


In [77]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 5ms/step - loss: 0.4785 - accuracy: 0.7913


[0.4784892797470093, 0.7913385629653931]

In [78]:
model_1_pred_probs=model_1.predict(val_sentences)

24/24 [==============================] - 0s 2ms/step


In [80]:
model_1_pred_probs.shape

(762, 1)

In [81]:
calculate_results(val_labels, model_1_pred_probs)

ValueError: ignored

In [82]:
val_labels

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,

In [85]:
#Convert Model_prediction Probabs into label formats
model_1_preds=tf.squeeze(tf.round(model_1_pred_probs))

In [86]:
model_1_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [87]:
model_1_results=calculate_results(y_true=val_labels,
                                  y_pred=model_1_preds)

In [88]:
model_1_results

{'accuracy': 79.13385826771653,
 'precision': 0.796653347825067,
 'recall': 0.7913385826771654,
 'f1-score': 0.7883318764387541}

In [89]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1-score': 0.7862189758049549}

In [91]:
import numpy as np
np.array(list(model_1_results.values()))> np.array(list(baseline_results.values()))

array([False, False, False,  True])

## Visualize the learned embeddings


In [93]:
#Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()

In [94]:
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [95]:
# Model 1 summary
model_1.summary()

Model: "Model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [100]:
#Get the weight matrix of the embedding layers
#These are numerical representation of our numerical datawhich are trained for 5 epochs

embed_weights=model_1.get_layer("embedding").get_weights()[0]
embed_weights

array([[-0.02625196, -0.02649532,  0.0208959 , ..., -0.01583313,
         0.01554727, -0.03784913],
       [ 0.01038581, -0.03351178, -0.02824967, ..., -0.02305223,
         0.00965059, -0.01141982],
       [-0.01091777,  0.05519261, -0.00579681, ...,  0.05836643,
         0.01102267,  0.01891178],
       ...,
       [ 0.04656248, -0.02546041, -0.04298569, ...,  0.01139244,
        -0.03442882,  0.02947399],
       [ 0.006305  ,  0.07467554,  0.03537539, ...,  0.00200303,
         0.06462538, -0.06900542],
       [ 0.11838141,  0.03921204,  0.04461341, ...,  0.09753927,
         0.02652887, -0.10442521]], dtype=float32)

In [101]:
print(embed_weights.shape)

(10000, 128)


Now we've got the embedding matrix our model has learned to represent tokens, let's see how we can visualize it

To do so, tensorflow has a tool called Projector

In [102]:
# # Code below is adapted from: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
import io

# Create output writers
out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# Write embedding vectors and words to file
for num, word in enumerate(words_in_vocab):
  if num == 0:
     continue # skip padding token
  vec = embed_weights[num]
  out_m.write(word + "\n") # write words to file
  out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
out_v.close()
out_m.close()

In [1]:
# Download files locally to upload to Embedding Projector
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download("embedding_vectors.tsv")
  files.download("embedding_metadata.tsv")

FileNotFoundError: ignored

## Recurrent Neural Networks(RNNs)

the premise of the Recurrent Neural Networs is to use the representation of the previous input to aid the representation of the later input

RNNs are used for sequence data (this example is a sequence of text)

 📖 Resources:

MIT Deep Learning Lecture on Recurrent Neural Networks - explains the background of recurrent neural networks and introduces LSTMs.
The Unreasonable Effectiveness of Recurrent Neural Networks by Andrej Karpathy - demonstrates the power of RNN's with examples generating various sequences.
Understanding LSTMs by Chris Olah - an in-depth (and technical) look at the mechanics of the LSTM cell, possibly the most popular RNN building block.